## 数据检索
用户输入只是问题，答案需要从已知的数据库或者文档中获取。因此，我们需要使用检索器获取上下文，并通过“question”键下的用户输入。

### RAG
检索增强生成（Retrieval-augmented Generation，RAG），是当下最热门的大模型前沿技术之一。如果将“微调（finetune）”理解成大模型内化吸收知识的过程，那么RAG就相当于给大模型装上了“知识外挂”，基础大模型不用再训练即可随时调用特定领域知识。

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings_path = "D:\\ai\\download\\bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)

C:\Users\a4154\AppData\Roaming\Python\Python310\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
d:\ProgramData\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
vectorstore = FAISS.from_texts(
    ["小明在华为工作","熊喜欢吃蜂蜜"],
    embedding=embeddings
)
vectorstore



In [3]:
#使用向量数据库生成检索器
retriever = vectorstore.as_retriever()

In [4]:
retriever.invoke("熊喜欢吃什么？")

[Document(page_content='熊喜欢吃蜂蜜'), Document(page_content='小明在华为工作')]

In [5]:
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)


In [6]:
from langchain_core.prompts import ChatPromptTemplate

template ="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [7]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

outputParser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {
        "context":retriever,
        "question":RunnablePassthrough()
    }
)

chain = setup_and_retrieval | prompt | model | outputParser

In [8]:
chain.invoke("小明在哪里工作？")

'小明在华为工作。'

In [9]:
#完整链式
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("小明在哪里工作？")

'小明在华为工作。'

In [18]:
from operator import itemgetter

template ="""
只根据以下文档回答问题：
{context}

问题：{question}
回答问题请加上称呼"{name}"。
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "name": itemgetter("name"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question":"小明在哪里工作？","name":"主人"})

'主人，小明在华为工作。'

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('./txt')

docs = loader.load()
docs